<a href="https://colab.research.google.com/github/maritaca-ai/maritalk-api/blob/add-notebook/examples/api/maritalk_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integrando a MariTalk com o LangChain

Esse notebook mostra como usar a MariTalk com o LangChain através de dois exemplos:

1. Exemplo simples de como usar a MariTalk para resolver uma tarefa.

2. LLM + RAG: O segundo exemplo mostra como responder uma pergunta cuja resposta se encontra em um documento longo, que não cabe no limite de tokens da MariTalk. Para isso iremos usar um buscador simples (BM25) para primeiramente buscar no documento os trechos mais relevantes e depois alimentá-los para a MariTalk responder.

## Instalando as bibliotecas necessárias

In [ ]:
!pip install maritalk -q
!pip install langchain[all] -q

# Implementação do MariTalk no LangChain

In [ ]:
from typing import Any, List, Mapping, Optional

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from langchain_core.language_models.chat_models import BaseChatModel, SimpleChatModel
from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
    AIMessage,
    BaseMessage,
)
import maritalk

class ChatMaritalk(SimpleChatModel):
    api_key: str
    temperature: float = 0.7
    chat_mode: bool = True
    max_tokens: int = 512
    do_sample: bool = True
    top_p: float = 0.95
    system_message_workaround: bool = True

    @property
    def _llm_type(self) -> str:
        return "maritalk"

    def parse_messages_for_model(self, messages: List[BaseMessage]):
        parsed_messages=[]

        for message in messages:
            if isinstance(message, HumanMessage):
                parsed_messages.append({"role":"user","content":message.content})
            elif isinstance(message, AIMessage):
                parsed_messages.append({"role":"assistant","content":message.content})
            elif isinstance(message, SystemMessage):
                if self.system_message_workaround:
                    parsed_messages.append({"role":"user","content":message.content})
                    parsed_messages.append({"role":"assistant","content": "ok"})

        return parsed_messages

    def _call(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        model = maritalk.MariTalk(key=self.api_key)
        stop_tokens=stop if stop is not None else []
        messages=self.parse_messages_for_model(messages)
        answer = model.generate(
            messages,
            temperature=self.temperature,
            max_tokens=self.max_tokens,
            stopping_tokens=stop_tokens,
            do_sample=self.do_sample,
            top_p=self.top_p,
            chat_mode=True,
        )

        return answer

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"system_message_workaround": self.system_message_workaround, "temperature": self.temperature, "top_p": self.top_p, "max_tokens": self.max_tokens }

# Exemplo 1 - Sugestão de animais de estimação

In [ ]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm=ChatMaritalk(
        api_key="",  # Insira aqui sua chave de API, que pode ser obtiva em chat.maritaca.ai -> "Chaves da API"
        temperature=0.7,
        chat_mode=False,
        max_tokens=100,
    )

output_parser = StrOutputParser()

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([
        ("system", """Você é uma assistente especialista em sugerir nome de animais de estimação. Dado o animal, você deve sugerir 4 nomes."""),
        ("human", """eu tenho um {animal}"""),
    ])

chain = chat_prompt | llm | output_parser

In [ ]:
response = chain.invoke({"animal": "cachorro"})
response

'1. Max\n2. Bella\n3. Charlie\n4. Rocky'

In [ ]:
response = chain.invoke({"animal": "peixe"})
response

'1. Nemo\n2. Dory\n3. Aqua\n4. Finny'

# Exemplo 2 - Sistema de perguntas e respostas do vestibular da UNICAMP 2024

Para este segundo exemplo, é necessario instalar algumas bibliotecas extras:

In [ ]:
!pip install unstructured rank_bm25 pdf2image pdfminer-six pikepdf pypdf unstructured_inference fastapi kaleido uvicorn "pillow<10.1.0" pillow_heif -q

## Carregando o edital

O primeiro passo é criar uma base de dados com as informações do edital. Para isso vamos baixar o edital do site da comvest e segmentar o texto extraído em janelas de 500 caracteres.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import OnlinePDFLoader

# Carregando o edital da comvest 2024
loader = OnlinePDFLoader("https://www.comvest.unicamp.br/wp-content/uploads/2023/10/31-2023-Dispoe-sobre-o-Vestibular-Unicamp-2024_com-retificacao.pdf")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100, separators=['\n', ' ', ''])
texts = text_splitter.split_documents(data)

## Criando um buscador

Agora que ja temos nossa base de dados, precisamos de um buscador. Para esse exemplo usaremos um simples BM25 como sistema de busca, mas este poderia ser substituido por qualquer outro buscador (como por exemplo busca via embeddings).

In [ ]:
from langchain.retrievers import BM25Retriever
retriever = BM25Retriever.from_documents(texts)

## Unindo Sistema Busca + LLM

Agora que ja temos nosso buscador, basta implementarmos um prompt especificando a tarefa e invocar a cadeia.

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts.prompt import PromptTemplate

prompt = """Baseado nos seguintes documentos, responda a pergunta abaixo.

{context}

Pergunta: {query}
"""

qa_prompt = ChatPromptTemplate.from_messages([("human", prompt)])

chain = load_qa_chain(llm, chain_type="stuff", verbose=True, prompt=qa_prompt)

In [ ]:
query = "Qual o tempo máximo para realização da prova?"

docs = retriever.get_relevant_documents(query)

chain.invoke({"input_documents": docs, "query": query})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 
Baseado nos seguintes documentos, responda a pergunta

§4º O(a) candidato(a) terá no máximo 5 (cinco) horas e no mínimo 2 (duas) horas para a realização das provas estabelecidas para cada dia. Poderá ser concedido tempo adicional aos(às) candidatos(as) nos casos previstos no art. 14.

§5º A ausência ou a obtenção de nota 0 (zero) em qualquer uma das provas, exceto nas provas de Habilidades Específicas, eliminará o(a) candidato(a) do VU 2024.

Art. 19 A 1ª fase será constituída de uma única prova de Conhecimentos Gerais composta por 72 (setenta e duas) questões objetivas sobre as áreas do conhecimento desenvolvidas no Ensino Médio, incluindo questões interdisciplinares.

§1º O(a) candidato(a) terá no máximo 5 (cinco) horas e no mínimo 2 duas horas para a realização da prova da 1ª fase. Poderá ser concedido tempo adicional aos(às) candidatos(as) nos casos previstos no art. 14

{'input_documents': [Document(page_content='§4º O(a) candidato(a) terá no máximo 5 (cinco) horas e no mínimo 2 (duas) horas para a realização das provas estabelecidas para cada dia. Poderá ser concedido tempo adicional aos(às) candidatos(as) nos casos previstos no art. 14.\n\n§5º A ausência ou a obtenção de nota 0 (zero) em qualquer uma das provas, exceto nas provas de Habilidades Específicas, eliminará o(a) candidato(a) do VU 2024.', metadata={'source': '/tmp/tmprm_mreyk/tmp.pdf'}),
  Document(page_content='Art. 19 A 1ª fase será constituída de uma única prova de Conhecimentos Gerais composta por 72 (setenta e duas) questões objetivas sobre as áreas do conhecimento desenvolvidas no Ensino Médio, incluindo questões interdisciplinares.\n\n§1º O(a) candidato(a) terá no máximo 5 (cinco) horas e no mínimo 2 duas horas para a realização da prova da 1ª fase. Poderá ser concedido tempo adicional aos(às) candidatos(as) nos casos previstos no art. 14.', metadata={'source': '/tmp/tmprm_mreyk/tmp